In [93]:
import pandas as pd
import numpy as np

import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import tensorflow
import keras
from tensorflow.keras.models import load_model

In [94]:
# load the model, scalers, encodings
model = load_model('model.keras')
with open('label_encoder_gender.pkl', 'rb') as f:
    gender = pickle.load(f)
with open('one_hot_encoder_geo.pkl', 'rb') as f: 
    geo = pickle.load(f)
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)



In [95]:
# example data
input_data = {
    'CreditScore': 502,
    'Geography': 'France',
    'Gender':'Female',
    'Age': 42,
    'Tenure': 8,
    'Balance': 156220,
    'NumOfProducts': 3,
    'HasCrCard': 2,
    'IsActiveMember': 1,
    'EstimatedSalary': 101348    
}

input_df = pd.DataFrame([input_data])
input_df
input_df['Gender'] = gender.transform(input_df['Gender'].map({'Male': 1, 'Female': 0}))
input_df
geo_encoded = geo.transform(input_df[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=geo.get_feature_names_out(['Geography']))
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [96]:
# concatenate the encoded geography columns with the original dataframe
input_df = pd.concat([input_df.drop(columns=['Geography']), geo_encoded_df], axis=1)


In [97]:
# scaler application
input_df_scaled = scaler.transform(input_df)



In [98]:
# predicition
predictions = model.predict(input_df_scaled)
predictions



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


array([[0.9818984]], dtype=float32)

In [99]:
predictions_prob = predictions[0][0]
predictions_prob

np.float32(0.9818984)

In [100]:
if predictions_prob > 0.5:
    print("Customer is predicted to leave the bank.")   
else:
    print("Customer is predicted to stay with the bank.")

Customer is predicted to leave the bank.
